This [Jupyter notebook](https://jupyter.org/)
is meant to be viewed in "slide mode".

If you want to view the slides locally
(with notebook version < 7),
you'll need to install [RISE](https://rise.readthedocs.io/).
With notebook version 7 and above,
and with JupyterLab,
you can try the [`jupyterlab_rise`](https://github.com/jupyterlab-contrib/rise) extension.

# Describing Three-Dimensional Movements in an Audio Scene Authoring Format

Matthias Geier

March 4<sup>th</sup>, 2024

Thesis:
[PDF](https://github.com/mgeier/diss/releases/download/initial-submission/Geier_Diss.pdf),
[LaTeX sources](https://github.com/mgeier/diss)  
Slides: https://github.com/mgeier/defense

### Spatial Audio Reproduction

<img src="loudspeaker-array.jpg" style="margin: 1em auto;" />

### Spatial Audio Reproduction

* binaural
* surround (5.1, 7.1, ..., 22.2)
* VBAP
* Ambisonics
* WFS
* ...

problem with "channel-based" audio reproduction:

* many output channels
* heterogeneous setups

solution:

* "object-based" audio reproduction

### Object-Based Audio Reproduction

* sound sources (mono signals)
* groups of sound sources
* observer position
* three-dimensional movement & trajectories
  * translation
  * rotation
  * movements in local coordinate systems, which are moving themselves
* dynamic volume control

### Demo (Browser GUI Prototype)

<img src="browser-gui-screenshot.png" style="margin: 1em auto;" />

### Classification of Object-Based Audio Formats

* sampled data
  * recorded audio data (e.g. sampled at 48000 Hz)
  * motion tracking data (e.g. sampled at 30 Hz)
* declarative
  * parametric (sparse) description of trajectories
  * synthesizer parameters
* procedural
  * script that creates position data in real time
  * graphical multimedia "patching" environment

### Example: X3D

a mix of categories within a single format:

* sampled data
  * source signals (conventional audio files) via `AudioClip` nodes
* declarative
  * trajectories (`PositionInterpolator`, `SplinePositionInterpolator`)
* procedural
  * animations via `Script` nodes

### Description of Movements in Existing Formats

* sampled data
  * SpatDIF, Spat-SDIF
* **declarative**
  * VRML, X3D, MPEG-4, Audio3D, XML3DAUDIO, ADM, TASCAR
* procedural
  * VRML, X3D, MPEG-4

### Interpolation of Positions and Orientations

* Audio3D, XML3DAUDIO (?)
  * linear interpolation for positions and orientations
* ADM
  * positions: linear; orientations: no interpolation
* TASCAR
  * positions: linear; orientations: Euler angles
  ([doesn't work](https://splines.readthedocs.io/en/0.3.1/rotation/naive-euler-angles-interpolation.html))
* VRML, MPEG-4
  * `PositionInterpolator` (linear interpolation),
    `OrientationInterpolator` (Slerp: spherical linear interpolation)
* X3D
  * `SplinePositionInterpolator` (Hermite & Catmull–Rom splines), `SquadOrientationInterpolator`

In [ ]:
%config InlineBackend.figure_formats = ['svg']

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import splines
from euclidean_helper import plot_spline_1d, plot_slopes_1d, grid_lines
from euclidean_helper import plot_tangents_2d, plot_tangent_2d

In [ ]:
from euclidean_helper import plot_spline_2d as _plot_spline_2d

def plot_spline_2d(*args, ax=None, **kwargs):
    if ax is None:
        ax = plt.gca()
    _plot_spline_2d(*args, ax=ax, **kwargs)
    ax.set_xlabel('x / m')
    ax.set_ylabel('y / m')

## Interpolation of Positions

In [ ]:
vertices = [
    (-2, -1),
    (-2, 0),
    (-0.1, 1.5),  # note: small distance from here ...
    (0.1, 1.5),   # ... to here.
    (2, 0),
    (3, 1),
]

In [ ]:
times = 0, 1, 2, 4, 5, 6  # note: larger interval in the middle!

In [ ]:
plot_spline_2d(splines.CatmullRom(vertices, grid=times))

### Centripetal Parameterization

* parameter interval is chosen to be the Euclidean distance between start and end of segment

In [ ]:
plot_spline_2d(splines.CatmullRom(vertices, alpha=0.5))

* curve shape only determined by vertices
* no independent control over time/speed

### Arc-Length Parameterization

* so far: good shape, but the timing is not controllable

* therefore: re-parameterization

  \begin{equation*}
  \boldsymbol{x}_\text{arc}(s) = \boldsymbol{x}(t(s))
  \end{equation*}

* arc-length parameterization $\to$ unit speed

  \begin{equation*}
  s(t) = \int\limits_{t_0}^t \left| \frac{d}{d\tau}\boldsymbol{x}(\tau) \right| d\tau
  \end{equation*}

* no analytic solution

* numerical integration, then find the inverse function with, e.g., the bisection method

### Arc-Length Parameterized Centripetal Catmull–Rom spline

In [ ]:
plot_spline_2d(splines.UnitSpeedAdapter(splines.CatmullRom(vertices, alpha=0.5)))

### More Re-Parameterization

* constant speed is a good default, but more control is needed
* therefore: another level of re-parameterization 
  based on [monotone splines](https://splines.readthedocs.io/en/0.3.1/euclidean/piecewise-monotone.html)
* time instants can be specified for any vertex
* speed (in m/s) can be specified (but with limits, to ensure monotonicity)

In [ ]:
plot_spline_2d(
    splines.NewGridAdapter(
        splines.UnitSpeedAdapter(splines.CatmullRom(vertices, alpha=0.5)),
        times))

### Position Splines, Summary

state of the art:

* cubic splines (Hermite and Catmull–Rom)

improvements:

* centripetal parameterization for more predictable curve shape
* re-parameterization for desired time/speed behavior
  * unit speed re-parameterization
  * monotone spline re-parameterization

## Interpolation of Orientations

state of the art, using
[unit quaternions](https://splines.readthedocs.io/en/0.3.1/rotation/quaternions.html)
to represent rotations:

* VRML, MPEG-4
  * `OrientationInterpolator` (Slerp: spherical linear interpolation)
* X3D
  * `SquadOrientationInterpolator` (Squad: spherical quadrangle interpolation)

### Representing Rotations With Unit Quaternions

quaternions:

* 4 components $\to$ 4-dimensional space $\mathbb{R}^4$
* quaternion multiplication is non-commutative ($ab \ne ba$)

unit quaternions:

* all quaternions with length 1 $\to$ unit hypersphere $S^3$
  * curved 3-dimensional space
* can be used to describe rotations (with "double cover")
* polynomials don't work
* interpolation along great circles (a.k.a. "Slerp")

In [ ]:
from quaternion_helper import angles2quat, plot_rotation, animate_rotations, display_animation

### Piecewise Slerp

In [ ]:
rotations = [
    angles2quat(0, 0, 180),
    angles2quat(0, 45, 90),
    angles2quat(90, 90, -80),
    angles2quat(90, 90, -90),
    angles2quat(-90, -45, 180),
]
times

In [ ]:
s_slerp = splines.quaternion.PiecewiseSlerp(
    rotations,
    grid=times,
    closed=True)
ani = animate_rotations(s_slerp.evaluate(
    np.linspace(s_slerp.grid[0], s_slerp.grid[-1], 200, endpoint=False)))
display_animation(ani, default_mode='loop')

### Using Lines to Construct Curves: De Casteljau's Algorithm

* control points at $\frac{1}{3}$ of the tangent vectors
* $\boldsymbol{x}_0$, $\boldsymbol{\tilde{x}}_0$,
  $\boldsymbol{\tilde{x}}_1$, $\boldsymbol{x}_1$:
  * 4 control points of a *cubic Bézier curve*

In [ ]:
s = splines.Monomial([[[-1.5, 2.75], [0, -4.5], [1.5, 0.75], [0, 1]]])

In [ ]:
def plot_tangents(s):
    plot_tangents_2d(s.evaluate(s.grid, 1), s.evaluate(s.grid))

In [ ]:
def plot_control_points(s):
    a = s.evaluate(0) + s.evaluate(0, 1) / 3
    b = s.evaluate(1) - s.evaluate(1, 1) / 3
    plt.scatter(*np.column_stack([a, b]), marker='x', color='black')
    plt.annotate(r'$\boldsymbol{x}_0$', s.evaluate(0) + [0, 0.1])
    plt.annotate(r'$\tilde{\boldsymbol{x}}_0$', a + [0, 0.1])
    plt.annotate(r'$\tilde{\boldsymbol{x}}_1$', b + [0, 0.1])
    plt.annotate(r'$\boldsymbol{x}_1$', s.evaluate(1) + [0, 0.1]);

In [ ]:
plot_spline_2d(s, chords=False)
plot_tangents(s)
plot_control_points(s)

### De Casteljau's Algorithm, Animated

In [ ]:
from casteljau import create_animation

def show_casteljau_animation(points, frames=30, interval=200):
    ani = create_animation(points, frames=frames, interval=interval)
    display({
        'text/html': ani.to_jshtml(default_mode='reflect'),
    }, raw=True)
    plt.close()  # avoid spurious figure display

In [ ]:
show_casteljau_animation([(0, 1), (0.5, 1.25), (1, 0), (0, 0)])

For rotations: replace linear interpolations with Slerp!

### Catmull–Rom-like Rotation Spline

In [ ]:
s_cr = splines.quaternion.CatmullRom(
    rotations,
    grid=times,
    endconditions='closed')
ani = animate_rotations(s_cr.evaluate(np.linspace(s_cr.grid[0], s_cr.grid[-1], 200, endpoint=False)))
display_animation(ani, default_mode='loop')

Chosen time intervals affect curve shape!

### Centripetal Parameterization

In [ ]:
s = splines.quaternion.CatmullRom(
    rotations,
    alpha=0.5,
    endconditions='closed')
ani = animate_rotations(s.evaluate(np.linspace(s.grid[0], s.grid[-1], 200, endpoint=False)))
display_animation(ani, default_mode='loop')

### Centripetal + Unit Speed Parameterization

In [ ]:
s = splines.UnitSpeedAdapter(splines.quaternion.CatmullRom(
    rotations,
    alpha=0.5,
    endconditions='closed'))

ani = animate_rotations(s.evaluate(np.linspace(s.grid[0], s.grid[-1], 200, endpoint=False)))

display_animation(ani, default_mode='loop')

### Why de Casteljau's algorithm instead of Squad?

De Casteljau provides derivative (i.e. angular speed),
which is needed to calculate unit speed parameterization.

![](de-casteljau.png)

Squad doesn't provide that.

### Centripetal + Unit Speed + Custom Parameterization

In [ ]:
s = splines.NewGridAdapter(
    splines.UnitSpeedAdapter(
        splines.quaternion.CatmullRom(
            rotations,
            alpha=0.5,
            endconditions='closed')),
    times)

In [ ]:
ani_times = np.linspace(s.grid[0], s.grid[-1], 200, endpoint=False)
ani = animate_rotations({
    'Catmull–Rom': s_cr.evaluate(ani_times),
    'Slerp': s_slerp.evaluate(ani_times),
    're-parameterized': s.evaluate(ani_times),
})
display_animation(ani, default_mode='loop')

### Rotation Splines, Summary

state of the art:

* Slerp and Squad

improvements:

* centripetal parameterization for more predictable curve shape
* re-parameterization for desired time/speed behavior
  * constant angular speed re-parameterization
    * de Casteljau's algorithm provides tangent vector (i.e. angular velocity)
  * monotone spline re-parameterization

## Audio Scene Description Format (ASDF)

* temporal relationships as primary structure
  * "time containers" inspired by the SMIL format
  * `<par>` and `<seq>`
    * can be arbitrarily nested
  * "time graph" instead of "scene graph"
* "clips" and "transforms" are positioned in the timeline
  * audio clips are stored as traditional audio files
  * transforms: position/rotation/volume splines
    * can be applied to
      * sound sources, or groups thereof
      * other transforms, or groups thereof
        * which can again be applied to ...
          * and so on ...

### Time Containers

`<par>` and `<seq>` inspired by the [SMIL](https://www.w3.org/TR/SMIL/) format:

```xml
<asdf version="0.4">
  <par>
    <clip file="audio/ukewave.ogg" pos="0 2" />
    <seq>
      <clip file="audio/marimba.ogg">
        <channel pos="-1 2" />
        <channel pos="1 2" />
      </clip>
      <clip file="audio/xmas.wav" pos="-1.5 0" />
    </seq>
  </par>
</asdf>
```

### Nested Transforms

```xml
<asdf version="0.4">
  <par>
    <clip id="ukulele" file="audio/ukewave.ogg" pos="-2 0" />
    <transform id="circular-motion" apply-to="ukulele" repeat="10">
      <o rot="0 0 0" />
      <o rot="0 0 90" />
      <o rot="0 0 180" />
      <o rot="0 0 -90" />
      <o rot="closed" />
    </transform>
    <transform id="forward-motion" apply-to="circular-motion">
      <o pos="0 -2" />
      <o pos="0 2" />
    </transform>
  </par>
</asdf>
```

### Mixed Transform Attributes

```xml
<asdf version="0.4">
  <par>
    <clip id="marimba" file="audio/marimba.ogg">
      <channel pos="-1 0" />
      <channel pos="1 0" />
    </clip>
    <transform apply-to="marimba">
      <o pos="0 -2" rot="-20" vol="1" />
      <o pos="0 0" time="1s" />
      <o vol="1" />
      <o rot="0" time="2s" />
      <o vol="0" />
      <o vol="1" time="65%" />
      <o pos="0 2" rot="20" vol="1"/>
    </transform>
  </par>
</asdf>
```

### Limitations of the ASDF

* audio only
* focus on movements
* deterministic scenes, no interaction
* no room simulation, no reverberation
* no trimming of audio clips, only integer repetitions
* no object rotation caused by the curvature of a trajectory
* ...

### Integration of the ASDF Library

* stand-alone spatialization app
  * SoundScape Renderer
  * integration with JACK transport
  
* plugin for multimedia framework
  * Pure Data (https://puredata.info/)
  * `asdf~`

### Research Output

* position and rotation splines
  * documentation: https://splines.readthedocs.io/ (incl. Python module `splines`)
  * splines library: https://github.com/AudioSceneDescriptionFormat/asdfspline-rust
    * implemented in Rust (providing Python language bindings)
* ASDF
  * spec with examples: https://AudioSceneDescriptionFormat.readthedocs.io/
  * more examples: https://github.com/AudioSceneDescriptionFormat/asdf-example-scenes
  * library: https://github.com/AudioSceneDescriptionFormat/asdf-rust
    * implemented in Rust (providing C language bindings)
    * incl. external for Pure Data: `asdf~` (implemented in C)
* SoundScape Renderer (incl. ASDF support): https://github.com/SoundScapeRenderer/ssr
* thesis: https://github.com/mgeier/diss
* slides: https://github.com/mgeier/defense

Everything is Open Source & Open Science!

## Bonus Slides

## Splines

Goal: scene author defines a few points in 3D space, a smooth curve is generated

further goals:

* don't stray too far from the control points
* changes in one place should only affect the immediate surroundings
* control timing, control speed

afterwards:

* same for rotation!

### Cubic Polynomial Curves (one-dimensional)

\begin{equation*}
\boldsymbol{p}_i(t) = \boldsymbol{d}_i t^3 + \boldsymbol{c}_i t^2 + \boldsymbol{b}_i t + \boldsymbol{a}_i
\end{equation*}

In [ ]:
def plot_slopes(s):
    grid = s.grid
    plot_slopes_1d(s.evaluate(grid, 1), s.evaluate(grid), grid, scale=2)

In [ ]:
s_x = splines.Monomial([[-1.5, 0, 1.5, 0]])
s_y = splines.Monomial([[2.75, -4.5, 0.75, 1]])

In [ ]:
plot_spline_1d(s_x); plot_slopes(s_x)
plot_spline_1d(s_y); plot_slopes(s_y)
plt.xlabel('t')
grid_lines([0, 1]); plt.ylim([-0.1, 1.5]);

### Cubic Polynomial Curves (two-dimensional)

\begin{equation*}
\boldsymbol{p}_i(t) = \boldsymbol{d}_i t^3 + \boldsymbol{c}_i t^2 + \boldsymbol{b}_i t + \boldsymbol{a}_i
\end{equation*}

In [ ]:
s1 = splines.Monomial([[[-1.5, 2.75], [0, -4.5], [1.5, 0.75], [0, 1]]])

In [ ]:
def plot_tangents(s):
    plot_tangents_2d(s.evaluate(s.grid, 1), s.evaluate(s.grid))

In [ ]:
plot_spline_2d(s1, chords=False)
plot_tangents(s1)

This also works for 3D and higher dimensions.

### Calculating Coefficients from Positions & Slopes

\begin{equation*}
\boldsymbol{p}_i(t) =
\left[\begin{matrix}t^{3} & t^{2} & t & 1\end{matrix}\right]
\left[\begin{matrix}2 & -2 & 1 & 1\\-3 & 3 & -2 & -1\\0 & 0 & 1 & 0\\1 & 0 & 0 & 0\end{matrix}\right]
\left[\begin{matrix}\boldsymbol{x}_{i}\\\boldsymbol{x}_{i+1}\\\boldsymbol{\dot{x}}_{i}\\\boldsymbol{\dot{x}}_{i+1}\end{matrix}\right]
\end{equation*}

### Smoothly Connecting Segments

* uniform Catmull–Rom splines
  (based on quadratic Lagrange interpolation and linear blending)
* between segments: incoming tangent equals outgoing tangent
* simple tangent equation:

  \begin{equation*}
  \boldsymbol{\dot{x}}_i = \frac{\boldsymbol{x}_{i+1} - \boldsymbol{x}_{i-1}}{2}
  \end{equation*}

In [ ]:
def plot_cr_tangent():
    points1 = [
        (-1, -0.5),
        (0, 2.3),
        (1, 1),
        (4, 1.3),
        (3.8, -0.2),
        (2.5, 0.1),
    ]
    idx = 2
    s1 = splines.CatmullRom(points1, endconditions='closed')
    plot_spline_2d(s1)
    plot_tangent_2d(
        s1.evaluate(s1.grid[idx], 1),
        s1.evaluate(s1.grid[idx]), color='purple')
    plt.plot(
        *s1.evaluate([s1.grid[idx - 1], s1.grid[idx + 1]]).T,
        '--', color='purple', linewidth=2)

In [ ]:
plot_cr_tangent()

### Cusps, Self-Intersections

* uniform parameter intervals (for each segment from 0 to 1)

In [ ]:
plot_spline_2d(splines.CatmullRom(vertices))

### Non-Uniform Splines

* arbitrary parameter interval for each segment

* tangent equation for uniform Catmull–Rom spline:

  \begin{equation*}
  \boldsymbol{\dot{x}}_i = \frac{\boldsymbol{x}_{i+1} - \boldsymbol{x}_{i-1}}{2}
  \end{equation*}
  
* tangent equation for non-uniform Catmull–Rom spline:

  \begin{equation*}
  \boldsymbol{\dot{x}}_i =
  \frac{
  (t_{i+1} - t_i) (\boldsymbol{x}_i - \boldsymbol{x}_{i-1})
  }{
  (t_i - t_{i-1})(t_{i+1} - t_{i-1})
  }
  +
  \frac{
  (t_i - t_{i-1}) (\boldsymbol{x}_{i+1} - \boldsymbol{x}_i)
  }{
  (t_{i+1} - t_i)(t_{i+1} - t_{i-1})
  }
  \end{equation*}

## Quaternions and Rotations

### (Unit) Quaternions

quaternions:

* 4-dimensional space
* 4 components, "scalar" and "vector" parts:
  \begin{equation*}
  q = (w, \vec{v}) = (w, (x, y, z))
  \end{equation*}
* multiplication is non-commutative

unit quaternions:

* all quaternions with length 1 $\to$ unit hypersphere $S^3$
* can be used to describe rotations (with "double cover")

### Unit Quaternions as Rotations

\begin{equation*}
q = \left(\cos \frac{\alpha}{2}, \vec{n} \sin \frac{\alpha}{2}\right)
\end{equation*}

In [ ]:
identity = angles2quat(0, 0, 0)
identity

In [ ]:
a = angles2quat(90, 0, 0)
b = angles2quat(0, 35, 0)
c = angles2quat(0, 0, 45)

In [ ]:
plot_rotation({
    'identity = 1': identity,
    '$a$': a,
    '$b$': b,
    '$c$': c,
});

### Quaternion Multiplication

non-commutative:

\begin{equation*}
q_m q_n \ne q_n q_m
\end{equation*}

In [ ]:
plot_rotation({'$ab$': a * b, '$ba$': b * a});

### Quaternion Multiplication

associative:

\begin{equation*}
(q_1 q_2) q_3 = q_1 (q_2 q_3)
\end{equation*}

In [ ]:
plot_rotation({'$(bc)a$': (b * c) * a, '$b(ca)$': b * (c * a)});

### Inverse

written as: $q^{-1}$

In [ ]:
plot_rotation({'$b$': b, '$b^{-1}$': b.inverse()});

\begin{equation*}
q q^{-1} = q^{-1} q = \boldsymbol{1}
\end{equation*}

for unit quaternions: $q^{-1} = \overline{q}$

\begin{equation*}
\overline{q}
= \left(w, -\vec{v}\right)
= \left(\cos \frac{\alpha}{2}, -\vec{n} \sin \frac{\alpha}{2}\right)
= \left(\cos \frac{-\alpha}{2}, \vec{n} \sin \frac{-\alpha}{2}\right)
\end{equation*}

### Exponentiation (integer exponents)

In [ ]:
plot_rotation({
    '$a^0 = 1$': a**0,
    '$a^1 = a$': a**1,
    '$a^2 = aa$': a**2,
    '$a^3 = aaa$': a**3,
});

### Exponentiation (real exponents)

In [ ]:
plot_rotation({
    '$a^1 = a$': a**1,
    '$a^{0.5}$': a**0.5,
    '$a^0 = 1$': a**0,
    '$a^{-0.5}$': a**-0.5,
});

\begin{equation*}
q^k = \left(\cos \frac{k\alpha}{2}, \vec{n} \sin \frac{k\alpha}{2}\right)
\end{equation*}

### Negation

In [ ]:
plot_rotation({'$c$': c, '$-c$': -c});

\begin{equation*}
-q
= \left(-w, -\vec{v}\right)
= \left(
\cos \frac{\alpha + 2 \pi}{2},
\vec{n} \sin \frac{\alpha + 2 \pi}{2}
\right)
\end{equation*}

### How To Interpolate Unit Quaternions?

* polynomials in a curved space?

* use De Casteljau's algorithm with [spherical linear interpolation (Slerp)](https://splines.readthedocs.io/en/0.3.1/rotation/slerp.html)
  * see the following slides

  \begin{equation*}
  \operatorname{Slerp}(q_0, q_1; t) = \left(q_1 {q_0}^{-1}\right)^t \, q_0
  \end{equation*}

### Slerp Animation

In [ ]:
from splines.quaternion import slerp

In [ ]:
q1 = angles2quat(45, -20, -60)
q2 = angles2quat(-45, 20, 30)

In [ ]:
ani_times = np.linspace(0, 1, 100)
slerp_ani = animate_rotations({
    'slerp(q1, q2)': slerp(q1, q2, ani_times),
    'slerp(q1, -q2)': slerp(q1, -q2, ani_times),
})

In [ ]:
display_animation(slerp_ani, default_mode='reflect')

### Calculating Bézier Control Points

Goal: find the Bézier control points
using Catmull–Rom tangents,
given neighboring rotations.

Based on the non-uniform
[equations for the Euclidean case](https://splines.readthedocs.io/en/0.3.1/euclidean/catmull-rom-non-uniform.html#Using-Non-Uniform-Bézier-Segments),
we are trying to come up with
[equations for rotations using unit quaternions](https://splines.readthedocs.io/en/0.3.1/rotation/catmull-rom-non-uniform.html)
(see next slide).

\begin{align*}
\boldsymbol{v}_i &= \frac{
\boldsymbol{x}_{i+1} - \boldsymbol{x}_i
}{
t_{i+1} - t_i
}
\\
\boldsymbol{\dot{x}}_i
&= \frac{
(t_{i+1} - t_i) \, \boldsymbol{v}_{i-1} + (t_i - t_{i-1}) \, \boldsymbol{v}_i
}{
t_{i+1} - t_{i-1}
}
\\
\boldsymbol{\tilde{x}}_i^{(+)}
&= \boldsymbol{x}_i + \frac{(t_{i+1} - t_i) \, \boldsymbol{\dot{x}}_i}{3}
\\
\boldsymbol{\tilde{x}}_i^{(-)}
&= \boldsymbol{x}_i - \frac{(t_i - t_{i-1}) \, \boldsymbol{\dot{x}}_i}{3}
\end{align*}

----

\begin{align*}
\vec{\rho}_{i} &= \frac{\ln(q_{i+1} {q_i}^{-1})}{t_{i+1} - t_i}
\\
\vec{\omega}_i &=
\frac{
(t_{i+1} - t_i) \, \vec{\rho}_{i-1} +
(t_i - t_{i-1}) \, \vec{\rho}_{i}
}{
t_{i+1} - t_{i-1}
}
\\
\tilde{q}_i^{(+)}
&=
\exp\left(\frac{t_{i+1} - t_i}{3} \, \vec{\omega}_i\right) \, q_i
\\
\tilde{q}_i^{(-)}
&=
\exp\left(\frac{t_i - t_{i-1}}{3} \, \vec{\omega}_i\right)^{-1} \, q_i
\end{align*}